# Tool - 1 Wikipedia Query Run 

In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_wrapper)

# Tool - 2 Nike Help Website scraping and Vector DB store

In [38]:
from langchain_community.document_loaders import WebBaseLoader, BSHTMLLoader
from langchain_core.documents import Document
from bs4 import BeautifulSoup
from selenium import webdriver
import tempfile, os

url = "https://www.nike.com/in/help/a/returns-policy"

driver = webdriver.Chrome()
driver.get(url)
html = driver.page_source
driver.quit()


with tempfile.NamedTemporaryFile(delete=False, suffix=".html") as f:
    f.write(html.encode("utf-8"))
    temp_path = f.name
    
with open(temp_path, "r", encoding="utf-8") as f:
    html = f.read()
    
soup = BeautifulSoup(html, 'html.parser')


In [44]:
text = soup.get_text()
docs = Document(page_content=text)

# web_loader = BSHTMLLoader(temp_path)
# docs = web_loader.load()
print(len(docs.page_content))

6485


In [52]:
import re

for tag in soup.select("nav, header, footer, #nav-menu, .nav"):
    tag.decompose()

text = soup.get_text(separator=' ')
text = re.sub(r"\n\s*\n+", "\n", text)
text = re.sub(r"[ \t]+", " ", text)
text = text.strip()
docs = Document(page_content=text)

# web_loader = BSHTMLLoader(temp_path)
# docs = web_loader.load()
print(len(docs.page_content))
docs.page_content


1900


"What is Nike's returns policy? | Nike Help \n GET HELP What can we help you with? What can we help you with? What is Nike's returns policy? Nike.com and Nike App purchases can be returned within 30 days ( some exceptions apply ). To be eligible to return, items must be in their original condition, unworn and unwashed with complete accessories, and the original packaging and tags must also be intact and in their original condition. This policy includes custom Nike By You sneakers. And remember, returns are always free for Nike Members—just make sure you sign in during checkout. \n So go ahead, shop with confidence. Shop Nike Start a Return Join Us FAQs How do I return a Nike order? You can start a return online. For additional details, refer to our simple instructions . And remember, we always cover the return delivery costs for Nike Members . Can I exchange a Nike order? The best way to exchange an order is to simply place a new order for the item you want and return the item you alre

In [53]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
splitted_docs = splitter.split_documents([docs])
print(splitted_docs[:1])

[Document(metadata={}, page_content="What is Nike's returns policy? | Nike Help")]


In [ ]:
! ollama pull embeddinggemma:latest

In [26]:
from langchain_community.vectorstores import FAISS
from langchain_ollama.embeddings import OllamaEmbeddings

db = FAISS.from_documents(splitted_docs, OllamaEmbeddings(model='embeddinggemma:latest'))
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000171D75CA180>, search_kwargs={})

In [27]:
from langchain_core.tools import tool

@tool
def retriever_tool(query: str):
    """Search for information from Nike Help page."""
    docs = retriever.invoke(query)
    return "\n\n".join(d.page_content for d in docs)

In [28]:
# Depreceated

# from langchain_classic.tools.retriever import create_retriever_tool
# retriever_tool = create_retriever_tool(retriever, "Nike Help Search", "Search for information from Nike Help page")
# retriever_tool

In [29]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=400)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=400))

# Agents

In [30]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [32]:

from langchain.agents import create_agent
from langchain_groq import ChatGroq

llm = ChatGroq(model='openai/gpt-oss-20b')
all_tools = [wiki_tool, retriever_tool, arxiv_tool]
prompt = """
You are a tool-using agent. Follow these rules strictly:

1. If the user asks anything about Nike, returns, orders, delivery, refunds, or shoes → USE ONLY `retriever_tool`.
2. If the user asks about a general topic → use `wikipedia`.
3. If the user asks about research papers → use `arxiv`.
4. Never make up tool arguments.
5. Never call a tool twice for the same purpose.
6. After receiving tool results → answer clearly and concisely.

"""

agent = create_agent(model=llm, tools=all_tools, system_prompt=prompt)

In [33]:
inputs = {"messages": [{"role": "user", "content": "What is Nike's returns policy for shoes?"}]}
response = agent.invoke(inputs, verbose=True)


In [ ]:
response

{'messages': [HumanMessage(content="What is Nike's returns policy for shoes?", additional_kwargs={}, response_metadata={}, id='33ef52cf-497c-4545-8978-618c0a1e8689'),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'User asks: "What is Nike\'s returns policy for shoes?" This is a question about Nike. According to rules: If the user asks anything about Nike, returns, orders, delivery, refunds, or shoes → USE ONLY retriever_tool. So we must call retriever_tool with query "Nike returns policy for shoes". Then give answer.', 'tool_calls': [{'id': 'fc_d73c23bb-c13f-4ff1-a50a-04d17b2cf5b3', 'function': {'arguments': '{"query":"Nike returns policy for shoes"}', 'name': 'retriever_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 99, 'prompt_tokens': 371, 'total_tokens': 470, 'completion_time': 0.10176796, 'completion_tokens_details': {'reasoning_tokens': 70}, 'prompt_time': 0.021066087, 'prompt_tokens_details': None, 'queue_time': 0.1888119

In [35]:
print(response['messages'])

[HumanMessage(content="What is Nike's returns policy for shoes?", additional_kwargs={}, response_metadata={}, id='33ef52cf-497c-4545-8978-618c0a1e8689'), AIMessage(content='', additional_kwargs={'reasoning_content': 'User asks: "What is Nike\'s returns policy for shoes?" This is a question about Nike. According to rules: If the user asks anything about Nike, returns, orders, delivery, refunds, or shoes → USE ONLY retriever_tool. So we must call retriever_tool with query "Nike returns policy for shoes". Then give answer.', 'tool_calls': [{'id': 'fc_d73c23bb-c13f-4ff1-a50a-04d17b2cf5b3', 'function': {'arguments': '{"query":"Nike returns policy for shoes"}', 'name': 'retriever_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 99, 'prompt_tokens': 371, 'total_tokens': 470, 'completion_time': 0.10176796, 'completion_tokens_details': {'reasoning_tokens': 70}, 'prompt_time': 0.021066087, 'prompt_tokens_details': None, 'queue_time': 0.188811922, 'total_time